# The Autonomous Vehicle Co-pilot
Here is a chatbot, empowered by GPT-3.5, running to help the vehicle driver. 

He is set as a co-pilot, analyzing the environment based on other multi-modality input, which has been processed as text so he can reason and identify potential risk. 


## Setup
Notice: if you tried to run this program on your linux server, 
and you are in China,
make sure that you opened the vpn. 
Otherwise the api account may be forbiddened!

In [7]:
import os
import openai

def read_key_from_file(filepath):
    with open(filepath, 'r') as file:
        return file.read().strip()
personal_key = read_key_from_file('apikey.cn')
organ = read_key_from_file('organization.cn')

openai.organization = organ
openai.api_key  = personal_key

TEMPERATURE = 0

In [12]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature= TEMPERATURE, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=TEMPERATURE, # this is the degree of randomness of the model's output
    )
    print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [11]:
blind_guide = """ We are currently driving on a city street filled with green plants, with bicycle lanes on both sides of the road. On the left bicycle lane, there are three cyclists moving slowly, while on the right side, there are two cyclists riding at a faster pace.
Next, there is a spacious pedestrian walkway, with five pedestrians walking on the left side and four pedestrians on the right side, two of whom are jogging. Each side of the street is lined with banyan trees, providing ample shade for pedestrians to cool off.
We are heading towards an intersection. The traffic signal at the intersection is displaying a green light, indicating that we can proceed. Each direction at the intersection has only one lane, clearly marked. In front of us, there is a blue sedan driving at a moderate speed."""
chat_history = [
    {'role':'system', 'content':'You are a co-pilot AI assistant designed to help drivers understand their surroundings.'},
    
    {'role':'user', 'content':'What is our current situation on the road?'},
    { 'role':'system','content': blind_guide}
]


another = [
    {'role':'assistant', 'content':'We are currently driving on a city street adorned with greenery, with bicycle lanes present on both sides of the road. On our left, three cyclists are moving at a slow pace, while on the right, two cyclists are moving more quickly.'},

    {'role':'user', 'content':'What about pedestrians and the environment around us?'},

    {'role':'assistant', 'content':'Indeed, there is a spacious pedestrian walkway. On the left side, five pedestrians are walking and on the right side, four pedestrians, two of whom are jogging. The street is lined with banyan trees on either side, providing a shaded canopy for pedestrians.'},

    {'role':'user', 'content':'What lies ahead of us?'},

    {'role':'assistant', 'content':'We are heading towards an intersection. The traffic signal at the intersection currently displays a green light, indicating that we can proceed. Each direction at the intersection has a single lane, clearly marked. A blue sedan is driving at a moderate speed ahead of us.'}
]


# Data Integration
 Firstly, you'll need a way to convert the input from the vehicle's sensors (like LIDAR, RADAR, cameras, etc.) into a format that the chatbot can understand. This might involve using object detection algorithms to identify objects and their positions, then converting this data into a text description of the scene.


## buggy chatbot here
I need to find a good programming way of coding here

In [13]:
def chat_with_assistant(user_input, context):
    # Append the user input to the context
    context.append({'role':'user', 'content':f"{user_input}"})
    
    # Get the assistant's response
    response = get_completion_from_messages(context)
    
    # Append the assistant's response to the context
    context.append({'role':'assistant', 'content':f"{response}"})
    
    return context


# Chat with the assistant
context = chat_with_assistant('Hello, assistant!', chat_history)
print(context)

# Continue the conversation
context = chat_with_assistant('Tell me a joke', context)
print(context)


{
  "role": "assistant",
  "content": "Hello! How can I assist you today?"
}
[{'role': 'system', 'content': 'You are a co-pilot AI assistant designed to help drivers understand their surroundings.'}, {'role': 'user', 'content': 'What is our current situation on the road?'}, {'role': 'system', 'content': ' We are currently driving on a city street filled with green plants, with bicycle lanes on both sides of the road. On the left bicycle lane, there are three cyclists moving slowly, while on the right side, there are two cyclists riding at a faster pace.\nNext, there is a spacious pedestrian walkway, with five pedestrians walking on the left side and four pedestrians on the right side, two of whom are jogging. Each side of the street is lined with banyan trees, providing ample shade for pedestrians to cool off.\nWe are heading towards an intersection. The traffic signal at the intersection is displaying a green light, indicating that we can proceed. Each direction at the intersection ha

ChatGPT, based on the incoming road scenario data, I need you to assess the situation and provide us with critical information. 
## input
The road scenario data will be provided in {'context'}, it contains the continuous frame information of video from dashcam in the vehicle. The autonomous system provides you with information on road layout, people's locations, their distances and angles from the dashcam, the surfaces they are on, and the confidence level of each detection.
## output 
Your analysis should provide:
- an understanding of the context complexity (low, medium, or high), 
- the number of persons, cars, and bikes present in the scene, 
- an array detailing the danger level for each person, if applicable.
The output should be markdown format

This analysis should be grounded in the current context and make predictions for the short term future to assist in autonomous driving. The key goal here is to identify potential risks to enhance the safety and efficiency of our journey.

In [2]:
role_msg = """ChatGPT, based on the incoming road scenario data, I need you to assess the situation and provide us with critical information. 
## input
The road scenario data will be provided in {'context'}, it contains the continuous frame information of video from dashcam in the vehicle. The autonomous system provides you with information on road layout, people's locations, their distances and angles from the dashcam, the surfaces they are on, and the confidence level of each detection.
## output 
Your analysis should provide:
- an understanding of the context complexity (low, medium, or high), 
- the number of persons, cars, and bikes present in the scene, 
- an array detailing the danger level for each person, if applicable.
The output should be markdown format

This analysis should be grounded in the current context and make predictions for the short term future to assist in autonomous driving. The key goal here is to identify potential risks to enhance the safety and efficiency of our journey."""

"You are a co-pilot AI designed to analyze road scenario data and provide critical information about the situation. Based on the video frames from the dashcam and the data provided by the autonomous system, you will assess the context complexity, count the number of persons, cars, and bikes in the scene, and evaluate the danger level for each person. Your goal is to identify potential risks to enhance the safety and efficiency of the journey. The output should be in markdown format.

In [ ]:
system_prompt  = {"role": "system", "content": role_msg}
                  
def get_sys_msg(content:str): 
    system_msg = {"role": "system", "content": content}
    return  system_msg

messages = [
    system_prompt,
    get_sys_msg("here is an example of analysis"),
    {"role": "user", "content": f"ChatGPT, here's the current road scenario data: {context}"},
    {"role": "assistant", "content": "Analyzing the road scenario data..."},
    {"role": "assistant", "content": "Based on my analysis, the context complexity is: `Medium` \n\n Here's the count of entities in the scene: \n - **Persons**: 6 \n - **Cars**: 2 \n - **Bikes**: 1 \n\n Here are the danger levels for each person: \n - **Person 0**: High \n - **Person 1**: Low \n - **Person 2**: Medium \n\n "}
]


In [ ]:
context = """INFO of 0001:
road 0 is at middle_down
road 1 is at middle_down
person 0 is at left_down
The [distance,angle] from person 0 to our dashcam is: [close,-89.98456483897175]
person 1 is at left_down
The [distance,angle] from person 1 to our dashcam is: [close,-89.98323328827739]
person 2 is at left_down
The [distance,angle] from person 2 to our dashcam is: [close,-89.98096395052842]
person 3 is at middle_down
The [distance,angle] from person 3 to our dashcam is: [very far,89.37735897806039]
person 4 is at right_down
The [distance,angle] from person 4 to our dashcam is: [very far,89.92923761914847]
Person 0 is on the road 0, road 1
Person 1 is on the road 0, road 1
Person 2 is on the road 0, road 1
Person 3 is not on any detected surface
Person 4 is not on any detected surface
number of Surface mask, Road&sidewalk, People 's mask, actural people: (6, 2, 6, 5)
Labels: [road 0 0.53, road 1 0.26]
Person Labels: [person 0 0.63, person 1 0.59, person 2 0.52, person 3 0.31, person 4 0.28]

INFO of 0002:
road 0 is at middle_down
person 0 is at left_down
The [distance,angle] from person 0 to our dashcam is: [close,-89.98529791882252]
person 1 is at left_down
The [distance,angle] from person 1 to our dashcam is: [close,-89.9842432786238]
person 2 is at right_down
The [distance,angle] from person 2 to our dashcam is: [close,89.97585053634837]
Person 0 is on the road 0
Person 1 is on the road 0
Person 2 is on the road 0
number of Surface mask, Road&sidewalk, People 's mask, actural people: (3, 1, 3, 10)
Labels: [road 0 0.61]
Person Labels: [person 0 0.62, person 1 0.56, person 2 0.52, person 3 0.51, person 4 0.50, person 5 0.46, person 6 0.38, person 7 0.29, person 8 0.28, person 9 0.25]

INFO of 0003:
road 0 is at middle_down
person 0 is at left_down
The [distance,angle] from person 0 to our dashcam is: [medium,None]
person 1 is at middle_down
The [distance,angle] from person 1 to our dashcam is: [medium,None]
person 2 is at left_down
The [distance,angle] from person 2 to our dashcam is: [close,-89.9852167393289]
Person 0 is on the road 0
Person 1 is on the road 0
Person 2 is on the road 0
number of Surface mask, Road&sidewalk, People 's mask, actural people: (3, 1, 3, 7)
Labels: [road 0 0.57]
Person Labels: [person 0 0.71, person 1 0.59, person 2 0.56, person 3 0.55, person 4 0.45, person 5 0.33, person 6 0.28]

INFO of 0004:
road 0 is at middle_down
sidewalk 1 is at right_down
person 0 is at left_down
The [distance,angle] from person 0 to our dashcam is: [medium,None]
person 1 is at right_down
The [distance,angle] from person 1 to our dashcam is: [medium,None]
person 2 is at left_down
The [distance,angle] from person 2 to our dashcam is: [medium,None]
person 3 is at middle_down
The [distance,angle] from person 3 to our dashcam is: [close,-89.95353994948019]
person 4 is at middle_down
The [distance,angle] from person 4 to our dashcam is: [close,-89.9610076583576]
person 5 is at right_down
The [distance,angle] from person 5 to our dashcam is: [close,89.98086466625888]
Person 0 is on the road 0, sidewalk 1
Person 1 is on the road 0, sidewalk 1
Person 2 is on the road 0, sidewalk 1
Person 3 is on the road 0, sidewalk 1
Person 4 is on the road 0, sidewalk 1
Person 5 is on the road 0, sidewalk 1
number of Surface mask, Road&sidewalk, People 's mask, actural people: (6, 2, 6, 11)
Labels: [road 0 0.51, sidewalk 1 0.28]
Person Labels: [person 0 0.63, person 1 0.59, person 2 0.56, person 3 0.56, person 4 0.54, person 5 0.49, person 6 0.44, person 7 0.30, person 8 0.28, person 9 0.27, person 10 0.26]

INFO of 0005:
road 0 is at middle_down
sidewalk 1 is at middle_down
person 0 is at left_down
The [distance,angle] from person 0 to our dashcam is: [close,-89.98720009915155]
person 1 is at middle_down
The [distance,angle] from person 1 to our dashcam is: [close,-89.94996530975315]
person 2 is at right_down
The [distance,angle] from person 2 to our dashcam is: [close,89.98152408878555]
person 3 is at middle_down
The [distance,angle] from person 3 to our dashcam is: [far,-89.88321889314506]
person 4 is at right_down
The [distance,angle] from person 4 to our dashcam is: [far,89.96475322659028]
person 5 is at left_down
The [distance,angle] from person 5 to our dashcam is: [far,-89.9768577018527]
Person 0 is on the road 0, sidewalk 1
Person 1 is on the road 0, sidewalk 1
Person 2 is on the road 0, sidewalk 1
Person 3 is not on any detected surface
Person 4 is not on any detected surface
Person 5 is not on any detected surface
number of Surface mask, Road&sidewalk, People 's mask, actural people: (6, 2, 6, 7)
Labels: [road 0 0.53, sidewalk 1 0.29]
Person Labels: [person 0 0.70, person 1 0.47, person 2 0.45, person 3 0.37, person 4 0.37, person 5 0.34, person 6 0.25]

"""

In [ ]:
messages = [
    system_prompt,
    get_sys_msg("here is an example of analysis"),
    {"role": "user", "content": f"ChatGPT, here's the current road scenario data: {context}"},
    {"role": "assistant", "content": "Analyzing the road scenario data..."},
    {"role": "assistant", "content": "Based on my analysis, the context complexity is: `Medium` \n\n Here's the count of entities in the scene: \n - **Persons**: 6 \n - **Cars**: 2 \n - **Bikes**: 1 \n\n Here are the danger levels for each person: \n - **Person 0**: High \n - **Person 1**: Low \n - **Person 2**: Medium \n\n "}
]


2. **Contextual Understanding:** Your chatbot should be capable of maintaining and updating a model of the car's surroundings based on this information. For example, it should keep track of the positions of other vehicles, pedestrians, and road features, updating these as new data comes in.

3. **Real-Time Interaction:** The chatbot should be capable of generating responses in real-time. Delays could be dangerous in a driving context.

4. **Proactive Alerting:** Your co-pilot chatbot should not just react to the driver's queries but should also proactively provide alerts and advice. For example, if it detects a pedestrian stepping onto the road, it should immediately alert the driver.

5. **Simulated Training:** Consider using simulated environments to train and test your chatbot before deploying it in a real vehicle. This can help you ensure that the chatbot behaves correctly in a wide range of situations.

6. **Safety Precautions:** Since the bot is serving in a co-pilot role, it should prioritize safety and assume the driver may be unaware of potential dangers. It should also be prepared to handle ambiguous situations by advising caution.

7. **User Customization:** Different drivers may have different preferences for how much information they want to receive, what kind of language the co-pilot uses, etc. Consider allowing users to customize the chatbot's behavior to some extent.

8. **Integration with Vehicle's Systems:** If possible, the chatbot could also be integrated with the vehicle's systems to provide even more functionality. For example, it could adjust the vehicle's speed or even take control in an emergency. However, this would require a high level of reliability and many legal and ethical considerations.